# Reconstructed Hits
Each `npz` file contains the following fields:
- `detector_curr`: **shape = (3, 9)**

  parameters of the current (misaligned) detector. We use 3 detectors for the linear dataset.
  The 9 numbers for each detector are center location (3), local x-direction (3), and local y-direction (3).
- `detector_start`: **shape = (3, 9)**

  parameters of the starting (ideal, not misaligned) detector. 
  The 9 numbers for each detector are center location (3), local x-direction (3), and local y-direction (3).
  We actually don't really need it now, since they are all the same for this toy dataset.
  The centers of the 3 detectors are (0, 10, 0), (0, 20, 0), and (0, 30, 0).
  The local x-direction are all (-1, 0, 0), and the local y-direction are all (0, 0, 1).
- `particle_vertex`: **shape = (50, 3)**

  the vertices of the trajectories.
- `particle_direction`: **shape = (50, 3)**

  the directon of the trajectories (NOT normalized).
- `recon_cc`: **shape = (50, 3, 3)**

  reconstructed hit locations in 3D space using parameters
  of the **misaligned** detectors on **misaligned** readout.
  `recon_cc[i, j]` is a 3D vector of the coordinates of particle `i` intersecting misaligned detector `j`.
  **NOTE:** Since the misaligned readout are produced exactly by the misaligned detectors,
  the reconstructed hits are perfect on the ground-truth trajectories with zero residual.
  Hence, it is the **ground-truth reconstructed hits**.
- `recon_pc`: **shape = (50, 3, 3)**

  reconstructed hit locations in 3D space using parameters
  of the **predicted** detectors on **misaligned** readout.
  `recon_pc[i, j]` is a 3D vector of the coordinates of particle `i` intersecting predicted detector `j`.
  **NOTE:** Since the predicted detectors is not perfect,
  the reconstructed hits will **NOT** have zero residual to the ground-truth trajectories.
  Let us call it **predicted reconsructed hits**.
- `closest_points`: **shape = (50, 3, 3)**

  The closest points on the ground-truth trajectories to the reconstructed hits by the **predicted** detector (`recon_pc`).
  **NOTE:** The residual I reported earlier is calculated as the mean squared error from the predicted reconstructed hits to the closest points.
  That means, the residual is **NOT** calculated as the MSE from the predicted reconstructed hits to the ground-truth reconstructed hits.

In [1]:
import numpy as np

## Let us load one data

In [16]:
fname = 'example_job/recon/sample_22.npz'
data = np.load(fname)

In [17]:
for key in data.keys():
    val = data[key]
    print(key, val.shape)

detector_curr (3, 9)
detector_start (3, 9)
particle_vertex (50, 3)
particle_direction (50, 3)
recon_cc (50, 3, 3)
recon_pc (50, 3, 3)
closest_points (50, 3, 3)


## calculate residual

In [19]:
residual = np.power(data['recon_pc'] - data['closest_points'], 2).mean()
print(residual)

0.00063665176


## per-hit per-coordinate difference

In [24]:
# between predicted reconstructed hits and closest points
particle_id = 3
detector_id = 0
diff = data['recon_pc'][particle_id, detector_id] - data['closest_points'][particle_id, detector_id]
# different in x direction
print(diff[0])

-0.00085502863


In [25]:
# between predicted reconstructed hits and ground-truth reconstructed hits
particle_id = 25
detector_id = 0
diff = data['recon_pc'][particle_id, detector_id] - data['recon_cc'][particle_id, detector_id]
# different in x direction
print(diff[0])

-0.011969686
